In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import classificacao

sns.set_style('darkgrid')
%matplotlib inline

In [3]:
df = pd.read_csv('conhecimento.csv')
df['dia'] = df.id_cena.str[-3:].astype(int)

id_cenas = [
    # 'LS82260702016078', 
    'LS82260702017256', # 226_070
    # 'LS82210702016059', 
    'LS82210702017269', # 221_070
    # 'LS52210672002068', 
    'LS82210672017285', # 221_067
    # 'LS82210742016059', 
    'LS82210742018128', # 221_074
]
col_sum = ['previsao', 'verdade', 'porcentagem', 'id_cena', 'caso']

In [4]:
def gera_teste_modelo_caso4(id_cena):
    print(id_cena)
    
    dia = int(id_cena[-3:])
    print(df.shape, dia, id_cena)

    teste_original = df[df.id_cena== id_cena].copy().reset_index(drop=True)
    teste = teste_original.copy()
    teste['verifica'] = 0
    print(len(teste_original))

    # Modelo
    op = teste.orb_pto.loc[0]
    print(op)
    modelo = df[(df.orb_pto == op)].copy()

    print(len(modelo))
    print(modelo.verifica.value_counts())

    return teste, teste_original, modelo

In [5]:
def get_id_cenas(df, orb_pto):
    return sorted(list(df[df.orb_pto==orb_pto].id_cena.unique()))
def resultado(teste, teste_original, verifica=1):
    r = pd.merge(teste[['id', 'classificacao']], teste_original[['id', 'verifica']], on='id')
    r['v'] = r.classificacao.astype(str) + r.verifica.astype(str)
    
    try:
        x = len(r[(r.verifica==verifica) & (r.classificacao == verifica)])
        v = len(r[r.verifica==verifica])
        p = int((x*100)/v)
    except:
        return [0, 0, 0]
    return [x, v, p]

In [6]:
colunas_stats = ['ndvi', 'nbrl', 'dif_ndvi', 'dif_dnbrl', 'medianb2', 'medianb3', 'medianb4', 'medianb5', 'medianb6', 'medianb7']

In [7]:
sumario = pd.DataFrame([], columns=col_sum)
for id_cena in id_cenas:
    teste, teste_original, modelo = gera_teste_modelo_caso4(id_cena)
    y_pred = classificacao.classifica(modelo, teste, colunas_stats)
    y_true = teste_original.sort_values('id').reset_index(drop=True)
    r = resultado(y_pred, y_true)
    r.append(id_cena)
    r.append(4)
    sumario = sumario.append(dict(zip(col_sum, r)), ignore_index=True)

    print(r)

LS82260702017256
(388551, 15) 256 LS82260702017256
961
226_070
25179
3    22397
1     2782
Name: verifica, dtype: int64
Criando classificador RF
RF - Classifica dado usando as colunas ['ndvi', 'nbrl', 'dif_ndvi', 'dif_dnbrl', 'medianb2', 'medianb3', 'medianb4', 'medianb5', 'medianb6', 'medianb7']
Classificando com NN
Cria avaliacao usando (RF+NN)/2 - Arredondado para 3 casas decimais
[580, 805, 72, 'LS82260702017256', 1]
LS82210702017269
(388551, 15) 269 LS82210702017269
801
221_070
62542
3    55291
1     7251
Name: verifica, dtype: int64
Criando classificador RF
RF - Classifica dado usando as colunas ['ndvi', 'nbrl', 'dif_ndvi', 'dif_dnbrl', 'medianb2', 'medianb3', 'medianb4', 'medianb5', 'medianb6', 'medianb7']
Classificando com NN
Cria avaliacao usando (RF+NN)/2 - Arredondado para 3 casas decimais
[291, 599, 48, 'LS82210702017269', 1]
LS82210672017285
(388551, 15) 285 LS82210672017285
1674
221_067
100057
3    55822
1    44235
Name: verifica, dtype: int64
Criando classificador RF
RF 

* Caso 1: Usando todo o conhecimento
* Caso 2: Quantidade de verifica 1 e 3 iguais
* Caso 3: Trimestre historico da orb_pto
> Em todos casos, o dado da própria passagem foi removido do conhecimento. E só esta sendo contabilizado dados de acerto de QUEIMADA

In [10]:
sumario

,previsao,verdade,porcentagem,id_cena,caso
0,580,805,72,LS82260702017256,4
1,291,599,48,LS82210702017269,4
2,738,1102,66,LS82210672017285,4
3,600,1072,55,LS82210742018128,4


In [13]:
sumario.rename(columns={'porcentagem': 'todas'}).to_csv('rascunho.csv', index=False)